In [2]:
import numpy as np
import matplotlib.pyplot as plt

### Generated norm distributed vector 'e'

In [13]:
p = 4
q = 3
n = p + q + 1
N = 150
eps = np.random.normal(0, 1, size=(N, ))
eps

array([ 0.04388137,  1.23955069,  0.16171555, -0.25848026,  1.06473909,
        0.03130247,  1.65944817,  0.93003921,  1.23065142, -0.05733287,
        0.7462489 , -0.05635537, -0.70536472, -1.37408238,  0.2049118 ,
        1.8581543 , -0.44883954, -0.14595323, -0.14823933,  0.06766581,
        0.53524872,  0.28720897,  0.19894946, -0.26184514, -0.08728815,
        0.68975839, -0.75047565, -0.99647507,  0.41391317,  0.32361215,
        0.85216019,  0.20091995,  0.94675284,  1.10367874,  1.70452088,
       -0.78362575, -1.82064658, -2.79930989, -1.65628161, -0.94057728,
       -0.23358473,  0.18846478,  1.26487377,  0.3702956 ,  0.19579923,
        0.87488213, -1.52312478,  0.89714388,  0.35218536,  0.68699852,
       -0.01229067,  1.85650581,  0.14909012,  0.23585811, -0.6837319 ,
       -0.19810994,  0.868864  ,  0.68353499,  0.30936919, -0.82899215,
       -0.51644206, -0.54560633, -0.25958559,  0.3038064 , -0.52559531,
       -0.04642289,  0.14431854,  1.0397942 , -0.13749486, -0.90

In [ ]:
v = np.random.normal()

### Generated 'y'

In [21]:
tetta = np.array([0.08, 0.25, -0.25, -0.19, 1, 0.7, 0.2, 0.1])
y = np.zeros((N,))

X = []
y[0] = eps[0]
y[1] = eps[1]
y[2] = eps[2]

for i in range(3, N):
    y[i] = tetta[0]  + tetta[1] * y[i - 1] + tetta[2] * y[i - 2] + tetta[3] * y[i - 3] + tetta[4] * eps[i - 1] + tetta[5] * eps[i - 2] + tetta[6] * eps[i - 3]
    
    X.append([1, y[i - 1], y[i - 2], y[i - 3], eps[i - 1], eps[i - 2], eps[i - 3]])

In [23]:
X = np.array(X)
X

array([[ 1.        ,  0.16171555,  1.23955069, ...,  0.16171555,
         1.23955069,  0.04388137],
       [ 1.        ,  0.16770292,  0.16171555, ..., -0.25848026,
         0.16171555,  1.23955069],
       [ 1.        , -0.17865579,  0.16770292, ...,  1.06473909,
        -0.25848026,  0.16171555],
       ...,
       [ 1.        ,  0.20706939,  0.65287795, ...,  0.26863486,
         0.30778989,  0.18532959],
       [ 1.        ,  0.00451515,  0.20706939, ...,  1.70053225,
         0.26863486,  0.30778989],
       [ 1.        ,  1.18019317,  0.00451515, ..., -0.15626258,
         1.70053225,  0.26863486]])

In [24]:
X.shape

(147, 7)

In [27]:
(X.T @ X).shape

(7, 7)

In [32]:
X.T.shape

(7, 147)

In [31]:
tetta_hat = ((np.linalg.inv(X.T @ X)) @ X.T)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 150 is different from 147)